In [ ]:
pip install selenium

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects, RequestException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import concurrent.futures

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def scrape_page(url):
    data = {}
    try:

        url = f'https://pbm-leb.com{url}'


        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-extensions')
        options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})


        browser = webdriver.Chrome(options=options)

        browser.get(url)

        # Wait for the page to load
        # Adjust the selector and timeout as needed
        wait = WebDriverWait(browser, 10)
        element_present = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'leafletmap-template-1')))

        # Scroll down incrementally
        # for i in range(10):  # Scroll down 10 times
        #     browser.execute_script("window.scrollBy(0, 800);")
        #     time.sleep(0.2)  # Adjust sleep time if needed

        # Get the rendered HTML
        html = browser.page_source

        data['source'] = url

        # Parse the HTML content
        soup = BeautifulSoup(html, 'html.parser')

        # header that contains the price and location string
        header = soup.find('div', attrs={'data-id': '2'})
        locationStr = header.find('h2')
        price = header.find('p', attrs={'class': 'price'})

        data['price'] = price.text.strip()
        data['location_str'] = locationStr.text.strip().split(" in ")[1]

        # body content details
        body = soup.find('div', attrs={'data-id': '4'})

        body_details = body.find('div', attrs={'id': 'details'})
        body_details_list = body_details.find_all('p')
        for detail in body_details_list:
            value = detail.find('span').text.strip()
            detail.find('span').decompose()
            key = detail.text.strip()
            data[key.replace(':', '')] = value

        # body content area data
        data['area'] = ''
        h3 = body.find('h3')
        ul = h3.find_next_sibling()
        lis = ul.find_all('li')
        for li in lis:
            value = li.text.strip()
            value = value.replace('- ', '')
            value = ' '.join(value.split())
            data['area'] += value + '; '

        # map coordinate url
        map = soup.find('div', {'class' : 'leafletmap-template-1'})
        img = map.find('img')
        data['map'] = img.get('src')

        # Proximities
        proximities = soup.find('div', {'class' : 'template-8'})
        data['proximities'] = proximities.text.strip()

        # Services
        service = body.find('div', {'class' : 'property-info-template-6'})
        services = service.find_all('li')
        data['services'] = ''
        for li in services:
            value = li.text.strip()
            data['services'] += value + '; '


        # info
        info = body.find('div', {'class' : 'info'})
        parag = info.find('p')
        data['info'] = parag.text.strip()

        # description
        description = soup.find('p', {'id' : 'description'})
        data['description'] = description.text.strip()
        print(data)
        return data
    finally:
        browser.quit()  # Close the browser even if an exception occurs

    # except TimeoutException:
    #     print("Timed out waiting for page to load")
    # except ConnectionError as e:
    #     print(f"Connection error occurred: {e}")
    # except Timeout as e:
    #     print(f"Timeout error occurred: {e}")
    # except TooManyRedirects as e:
    #     print(f"Too many redirects: {e}")
    # except RequestException as e:
    #     print(f"An error occurred RequestException: {e}")
    # except Exception as e:
    #     print(f"An error occurred: {e}")

    # return None

In [ ]:
def scraping_data(path, file_name):
    # Load the CSV file into a DataFrame
    urls = pd.read_csv(path + file_name)
    print(len(urls))
    all_data = []
    failed_url = []
    index = 2
    def process_url(url):
        print(f"Processing {url}")
        return scrape_page(url)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map the process_url function to the list of URLs
        future_to_url = {executor.submit(process_url, url['URL']): url for index, url in urls.iterrows()}

        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                page_data = future.result()
                all_data.append(page_data)
                index += 1
                print(index)
            except Exception as exc:
                print(f"{url['URL']} generated an exception: {exc}")
                failed_url.append(url['URL'])

    # Filter out None values
    filtered_data = [item for item in all_data if item is not None]

    # Create DataFrame
    df = pd.DataFrame(filtered_data)

    csv_file_path = f'{path}data_{file_name}'
    df.to_csv(csv_file_path, index=False)
    return all_data, failed_url

In [6]:
# Replace the path with your file's path
# file_path_rent = '/content/drive/MyDrive/M2 final project/PBM website/'
file_path_sale = '/content/drive/MyDrive/M2 final project/PBM website/'
d,f = scraping_data(file_path_sale, 'urls_to_be_scraped_sale.csv')
# scraping_data(file_path_rent, 'urls_to_be_scraped_rent.csv')

Output hidden; open in https://colab.research.google.com to view.